In [1]:
from __future__ import division
%matplotlib inline
import csv
import sys
import os
import traceback
import json
import numpy as np
import pandas as pd
from tabulate import tabulate
from pandas.tools.plotting import lag_plot, autocorrelation_plot
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
from scipy.stats import norm, skew, kurtosis
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AR
import scipy.fftpack as fft

from helper_functions import getListOfFiles, getCSV, getStatistics, remap_interval

## Exploring meta-features

In [2]:
csv_dir = "./data"
files = getListOfFiles(csv_dir, ".csv")
print ">> {} CSV Files found:\n".format(len(files))
print json.dumps(files)

>> 34 CSV Files found:

["_2016-11-23-18-49-13_exp1_Player.csv", "_2016-11-23-18-49-13_exp2_Player.csv", "_2016-11-23-18-49-13_exp3_Player.csv", "_2016-11-23-18-49-13_exp4_Player.csv", "_2016-11-23-18-49-13_exp5_Player.csv", "_2016-11-24-15-43-37_exp1d_Player.csv", "_2016-11-24-15-43-37_exp2d_Player.csv", "_2016-11-24-15-43-37_exp3d_Player.csv", "_2016-11-24-15-43-37_exp4d_Player.csv", "_2016-11-24-15-43-37_exp5d_Player.csv", "_2016-11-24-15-43-37_exp6d_Player.csv", "_2016-11-24-16-23-29_expa_Player.csv", "_2016-11-24-16-23-29_expb_Player.csv", "_2016-11-24-16-23-29_expc_Player.csv", "_2016-11-24-16-23-29_expd_Player.csv", "_2016-11-24-16-48-48_exp1d_Player.csv", "_2016-11-24-16-48-48_exp2d_Player.csv", "_2016-11-24-16-48-48_exp3d_Player.csv", "_2016-11-24-17-15-38_expa_Player.csv", "_2016-11-24-17-15-38_expb_Player.csv", "_2016-11-24-17-15-38_expc_Player.csv", "_2016-11-24-17-40-06_expb_Player.csv", "_2016-11-26-15-42-51_exp1d_Player.csv", "_2016-11-26-16-05-47_exp1d_Player.csv", "_20

Now, we define a dictionary containing a remap of the name of the features. This is to easy visualization since ros feature names may be very long and also allow quick decoupled modifications (just edit the variable locally). Also, from it we define a ignore list containing the names we do not want to consider for the analysis. That is done by setting to the ignore list all dictionary keys with empty values. **Here we assume the names are consistent feature name w.r.t. the `csv` data. Otherwise, a `ValueError` exception is likely to be thrown!**

In [3]:
# variable for storing the loaded feature names.
feature_name_map = {
  "time" : "time",
  "Control": "control",
  "High_level": "high_level",
  "Expectation": "expectation",
  "Activity": "activity",
  "/kinect_features/.ci": "ci",
  "/kinect_features/.distance": "distance",
  "/kinect_features/.proximity": "proximity",
  "robogame/imu_state.gyro.x": "gyroX",
  "robogame/imu_state.gyro.y": "gyroY",
  "robogame/imu_state.gyro.z": "gyroZ",
  "robogame/imu_state.linear_acc.x": "accX",
  "robogame/imu_state.linear_acc.y": "accY",
  "robogame/imu_state.linear_acc.z": "accZ"
}

ignore_col_list = [k for k,v in feature_name_map.items() if v is ""]
print "List of ignored topics: {}".format(ignore_col_list)

List of ignored topics: []


In [164]:
def loadFile(file_name, verbose=False):
    ##NOTE: IF "TOO MANY VALUES TO UNPACK" ERROR IN THE getCSV METHOD, RESTART THE KERNEL. SOMETHING MUST BE WRONG WITH
    # THE KERNEL INITIALIZATION. MUST BE CHECKED! (LOW-PRIORITY)

    csv_data = None          # the variable where the loaded csv data is stored.
    num_windows = 0          # the number of windows loaded.
    windows = []             # the list of windows data. Each element is a pandas dataframe 
                             #  corresponding to the windows. The list is of size 'num_windows'.

    print '-- Processing: "{}"'.format(file_name)

    # load the data, abort in case of error.
    try:
        num_windows, csv_data = getCSV(os.path.join(csv_dir, file_name))
    except ValueError as e:
        print traceback.format_exc()
        sys.exit(-1)

    for w in range(num_windows):
        win_data = {}
        for k in csv_data.keys():
            # consider the data only if it is not in the ignore list.
            if k not in ignore_col_list:
                if  csv_data[k][w] == []:
                    raise ValueError("\tFile has empty tagged windows. Skipping...")
                win_data[feature_name_map[k]] = csv_data[k][w]
                
        # convert dictionary to dataframe and save it to list of all windows data for the file.
        windows.append(pd.DataFrame.from_dict(win_data))
    
    print '-- Retrieved {} windows in {}'.format(num_windows, file_name)
    
    if verbose:
        overlap_reference = 50
        try:
            _, n_windows, sample_info, avg_overlap, avg_diff = getStatistics(csv_data, compareWith=overlap_reference)
            print "LOAD SUMMARY:"
            print tabulate([[n_windows,"{:.2f}".format(avg_overlap),"{:.2f}".format(avg_diff)]],
                               headers=["#Win", "Avg. Overlap", "Avg. dev. from ref."])
        except ValueError as e:
            print traceback.format_exc()
            return None
    return windows

## Auxiliary functions

In [118]:
def flatten_dict(dd, separator='.', prefix=''):
    """This function collapses a dictionary into a list, by appending
    the keys' values to themselves. That is, parents(keys) are joined together
    with children (values) by the separator variable.
    dd  :   dictionary to be flattened
    separator   :   the character used to join values to their keys.
    prefix      :   the character used in place of the value.
    """
    return {prefix + separator + k if prefix else k: v
            for kk, vv in dd.items()
            for k, v in flatten_dict(vv, separator, kk).items()
            } if isinstance(dd, dict) else {prefix: dd}

In [ ]:
class MetafeatureRow():
    """Upon call to store, organize the data in 
    a dictionary. Convenient for using with csv.DictWriter."""
    def __init__(self):
        self.rows = defaultdict(dict)

    def store(self,d,func):
        """Each new value of a function is stored in a list
        for the given attibute."""
        for k,v in d.items():
            self.rows[k][func] = float("{:.2f}".format(v))
        
    def getData(self):
        """Return dictionary where each key is the atribute
        and its value a dictionary containing the metafeatures
        calculated"""
        return self.rows

## Meta-features function definition

In [163]:
def mean(data):
    """Calculates mean of the data"""
    if isinstance(data,list):
        return [d.mean(axis=0, skipna=True, numeric_only=True).round(2) for d in data]
    else:
        return data.mean(axis=0, skipna=True)


def std(data):
    """Calculates the standard deviation"""
    if isinstance(data,list):
        return [d.std(axis=0, skipna=True, numeric_only=True) for d in data]
    else:
        return data.std(axis=0, skipna=True, numeric_only=True)


def max_value(data):
    """ Calculates Largest value in array"""
    if isinstance(data,list):
        return [d.max(axis=0, skipna=True, numeric_only=True) for d in data]
    else:
        return data.max(axis=0, skipna=True, numeric_only=True)

    
def min_value(data):
    """Calculates smallest value in array"""
    if isinstance(data,list):
        return [d.min(axis=0, skipna=True, numeric_only=True) for d in data]
    else:
        return data.min(axis=0, skipna=True, numeric_only=True)


def mad(data):
    """ Calculates the median absolute deviation"""
    if isinstance(data,list):
        l = []
        for d in data:
            m = {}
            for k in data._get_numeric_data():
                m[k] = abs(data[k].dropna() - data[k].median())
                m[k] = pd.Series(m[k]).median()
            l.append(pd.Series(m))
        return l
    else:
        m = {}
        for k in data._get_numeric_data():
            m[k] = abs(data[k].dropna() - data[k].median())
            m[k] = pd.Series(m[k]).median()
        return pd.Series(m)


def sma(data):
    """Computes Signal magnitude area.
    http://dsp.stackexchange.com/questions/18649/signal-magnitude-area
    """
    if isinstance(data,list):
        return [d.sum(axis=0, skipna=True, numeric_only=True) / data.shape[0] for d in data]
    else:
        return data.sum(axis=0, skipna=True, numeric_only=True) / data.shape[0]


def energy(data):
    """Energy measure. Sum of the squares divided by the number of values."""
    for d in data:
        row = data[np.isfinite(data[d])].apply(lambda x: x**2).mean(axis=0, skipna=True)
    return pd.Series(row)

def iqr(data):
    """Calculates the interquartile range
    http://stackoverflow.com/questions/23228244/how-do-you-find-the-iqr-in-numpy
    """
    if isinstance(data,np.ndarray):
        return np.subtract(*np.percentile(data, [75, 25]))
    else:
        v = {}
        for k in data._get_numeric_data():
            v[k] = np.subtract(*np.percentile(data[k].dropna(), [75, 25]))
        return pd.Series(v)

def entropy(data):
    """Signal entropy"""
    pass

def maxInds(data, n_bins=200, filterMean= True):
    """Returns the index of the frequency component with largest magnitude"""
    m_indexes = {}
    for k in data._get_numeric_data():
        c_sig = []
        if filterMean:
            filtered = data[k].dropna().as_matrix()
            mean_sig = np.ones_like(filtered)*np.mean(filtered)
            # remove mean of the signal, for better results.
            c_sig = data[k].dropna().as_matrix() - mean_sig
        freqsig = fft.fft(c_sig,n=n_bins) 
        half_freq_domain = freqsig[:int(n_bins/2)]
        #get index in the freq domain
        m_indexes[k] = np.where(np.abs(half_freq_domain)==(max(np.abs(half_freq_domain))))[0][0]
    return pd.Series(m_indexes)

def meanFreq(data, n_bins=200, filterMean=True):
    """
    Weighted average of the frequency components to obtain a mean frequency
    http://luscinia.sourceforge.net/page26/page35/page35.html
    """
    m_freq = {}
    for k in data._get_numeric_data():
        c_sig = []
        if filterMean:
            filtered = data[k].dropna().as_matrix()
            mean_sig = np.ones_like(filtered)*np.mean(filtered)
            # remove mean of the signal, for better results.
            c_sig = data[k].dropna().as_matrix() - mean_sig
        freqsig = fft.fft(c_sig,n=n_bins) 
        half_freq_domain = freqsig[:int(n_bins/2)]
        #get index in the freq domain
        m_freq[k] = np.sum(np.abs(half_freq_domain) * range(len(half_freq_domain))) / sum(np.abs(half_freq_domain))
    return pd.Series(m_freq)

def skewness(data, n_bins=200, filterMean=True): 
    """skewness of the frequency domain signal"""
    m_skew = {}
    for k in data._get_numeric_data():
        c_sig = []
        if filterMean:
            filtered = data[k].dropna().as_matrix()
            mean_sig = np.ones_like(filtered)*np.mean(filtered)
            # remove mean of the signal, for better results.
            c_sig = data[k].dropna().as_matrix() - mean_sig
        freqsig = fft.fft(c_sig,n=n_bins) 
        half_freq_domain = freqsig[:int(n_bins/2)]
        #get index in the freq domain
        m_skew[k] = skew(c_sig)
    return pd.Series(m_skew)

def kurtos(data, n_bins=200, filterMean=True):
    """kurtosis of the frequency domain signal"""
    m_kurtosis = {}
    for k in data._get_numeric_data():
        c_sig = []
        if filterMean:
            filtered = data[k].dropna().as_matrix()
            mean_sig = np.ones_like(filtered)*np.mean(filtered)
            # remove mean of the signal, for better results.
            c_sig = data[k].dropna().as_matrix() - mean_sig
        freqsig = fft.fft(c_sig,n=n_bins) 
        half_freq_domain = freqsig[:int(n_bins/2)]
        #get index in the freq domain
        m_kurtosis[k] = kurtosis(c_sig)
    return pd.Series(m_kurtosis)

def bandsEnergy():
    """Energy of a frequency interval within the bins of the FFT."""
    pass

def angle():
    """Angle between to vectors."""
    pass

def arCoeff(): 
    """Autorregresion coefficients with Burg order equal to 4"""
    pass

def correlation_acc(data): 
    """correlation coefficient between two accelerometer signals"""
    cor = data[["accY", "accX", "accZ", "gyroZ", "gyroX", "gyroY"]].corr().to_dict()['accY']
    res = {}
    for k,v in cor.iteritems():
        if k == 'accY':
            continue
        res[k+'-accY'] = v
    return res

def correlation_kinect(data): 
    """correlation coefficient between two accelerometer signals"""
    cor = flatten_dict(data[["ci","proximity"]].corr().to_dict()["ci"])
    res = {}
    for k,v in cor.iteritems():
        if k == 'ci':
            continue
        res[k+'-ci'] = v
    return res

In [173]:
function_dispatcher = {
    "mean"     : mean,
    "std"      : std,
    "max"      : max_value,
    "min"      : min_value,
    "mad"      : mad,
    "sma"      : sma,
    "iqr"      : iqr,
    "energy"   : energy,
    "maxInds"  : maxInds,
    "meanFreq" : meanFreq,
    "skewness" : skewness,
    "kurtosis" : kurtos
}

## Testing on just a single window

In [169]:
class tabularData():
    def __init__(self):
        self.rows = defaultdict(list)
        self.headers = ['topics']

    def store(self,d,func):
        """Each new value of a function is stored in a list
        for the given attibute."""
        for k,v in d.items():
            self.rows[k].append(float("{:.2f}".format(v)))
        self.headers.append(func)
        
    def getData(self):
        r = []
        for k,v in self.rows.items():
            r.append([k] + [i for i in v])
        return sorted(r)
    
    def getHeaders(self):
        return self.headers

try:
    data = loadFile(files[0])[0]._get_numeric_data()
except ValueError as e:
    print traceback.format_exc()


table_rows = tabularData()
table_rows.store(mean(data).to_dict(), "mean")
table_rows.store(std(data).to_dict(), "std")
table_rows.store(max_value(data).to_dict(), "max")
table_rows.store(min_value(data).to_dict(), "min")
table_rows.store(mad(data).to_dict(), "mad")
table_rows.store(sma(data).to_dict(), "sma")
table_rows.store(iqr(data).to_dict(), "iqr")
table_rows.store(energy(data).to_dict(), "energy")
table_rows.store(maxInds(data).to_dict(), "maxInds")
table_rows.store(meanFreq(data).to_dict(), "meanFreq")
table_rows.store(skewness(data).to_dict(), "skewness")

table_rows2 = tabularData()
table_rows2.store(kurtos(data).to_dict(), "kurtosis")

print 115 * '*'
print 45*' '+"META-FEATURES - 1 windows"
print 115 * '*'
print    
print tabulate(table_rows.getData(), headers=table_rows.getHeaders(), numalign="right")
print 115 * '-'
print tabulate(table_rows2.getData(), headers=table_rows2.getHeaders(), numalign="right")

-- Processing: "_2016-11-23-18-49-13_exp1_Player.csv"
-- Retrieved 24 windows in _2016-11-23-18-49-13_exp1_Player.csv
*******************************************************************************************************************
                                             META-FEATURES - 1 windows
*******************************************************************************************************************

topics         mean      std    max    min    mad      sma      iqr       energy    maxInds    meanFreq    skewness
---------  --------  -------  -----  -----  -----  -------  -------  -----------  ---------  ----------  ----------
accX       -1071.07  1946.72   2293  -6422   1610  -852.25  3328.25  4.91132e+06          4       17.64       -0.28
accY         428.97   620.15   2006  -1131  354.5   341.33   717.75       566011          3       25.16       -0.17
accZ        2056.84  1609.09   8636  -3187    701  1636.62   1335.5  6.80225e+06         12       27.07        1.0

## Get meta features for all windows and csv files

In [176]:
def get_metadata(data, target="control", all_targets=['control','expectation','high_level','activity'],):
    """Export the metadata to a file
    data : the list of windows data.
    target   :   the target we want to export
    """
    numeric_cols = data[0]._get_numeric_data().columns.values
    exclude = list(set(['control','expectation','high_level','activity']) - set(['control'])) + ['time']
    
    X_output_filename = open("{}_X.csv".format(target), 'wa')
    y_output_filename = open("{}_y.csv".format(target), 'wa')
        
    Xwriter = csv.DictWriter(X_output_filename, [k for k in numeric_cols if k not in exclude])
    # write the headers
    Xwriter.writeheader()
    # flush data
    X_output_filename.flush()
    ywriter = csv.writer(y_output_filename)
    ywriter.writerow([target])
    for df in data:
        rows = mean(df).to_dict()
        del rows["time"]
        Xwriter.writerows([rows])   #write content to the file
        #Xwriter.writerows([{}])    #write an empty line to mark the end of the windows
        X_output_filename.flush()   #flush data.
        ywriter.writerow([df[target][0]])
        y_output_filename.flush()
    

def getSupervisedMetaFeatures(listOfFiles, target="control", all_targets=['control','expectation','high_level','activity'],
                     mf=["mean","std","max","min","mad","sma","iqr","energy","maxInds","meanFreq","skewness","kurtosis"]):
    """Export the metadata to a file
    data : the list of windows data.
    target   :   the target we want to export
    """
    
    exclude = list(set(all_targets) - set(target)) + ['time']
    X_output_filename = None
    y_output_filename = None
    Xwriter = None
    ywriter = None
    data = []
    failed_files = []
    
    for i, csv_filename in enumerate(listOfFiles):
        try:
            file_data = loadFile(csv_filename)
        except ValueError as e:
            print traceback.format_exc()
            failed_files.append(csv_filename)
            continue
        
        if i == 0:
            numeric_cols = file_data[0]._get_numeric_data().columns.values
            
            attributes = dict.fromkeys([k for k in numeric_cols if k not in exclude])
            for k in attributes.keys():
                attributes[k] = dict.fromkeys(mf)
        
            X_output_filename = open("{}_X.csv".format(target), 'wa')
            #y_output_filename = open("{}_y.csv".format(target), 'wa')

            Xwriter = csv.DictWriter(X_output_filename, flatten_dict(attributes,separator='_').keys() + [target])
            # write the headers
            Xwriter.writeheader()
            # flush data
            X_output_filename.flush()
            #ywriter = csv.writer(y_output_filename)
            #ywriter.writerow([target])
            
        for win_df in file_data:
            win_df_numeric = win_df._get_numeric_data()
            meta_row = MetafeatureRow()
            
            for f in mf:
                try:
                    func = function_dispatcher[f]
                    meta_row.store(func(win_df_numeric.drop('time', 1)).to_dict(), f)
                except KeyError:
                    raise ValueError('Invalid function: {}'.format(f))

            rows = flatten_dict(meta_row.getData(),separator='_')
            rows[target] = win_df[target][0]
            Xwriter.writerows([rows])   #write content to the file
            X_output_filename.flush()   #flush data.
            #ywriter.writerow([win_df[target][0]])
            #y_output_filename.flush()
        
        ### write an empty line to mark the end of the file
        Xwriter.writerows([{}])
        X_output_filename.flush()
        #ywriter.writerows([""])
        #y_output_filename.flush()
        print "Meta-features saved to file!"
        print
        #############################
    X_output_filename.close()
    print '-- List of failed files:\n{} '.format(json.dumps(failed_files, indent=4))

## Export meta-features

In [177]:
getSupervisedMetaFeatures(files, target="activity", mf=["mean","std","max","min","mad","sma","iqr","maxInds","skewness","kurtosis"])

-- Processing: "_2016-11-23-18-49-13_exp1_Player.csv"
-- Retrieved 24 windows in _2016-11-23-18-49-13_exp1_Player.csv
Meta-features saved to file!

-- Processing: "_2016-11-23-18-49-13_exp2_Player.csv"
-- Retrieved 28 windows in _2016-11-23-18-49-13_exp2_Player.csv
Meta-features saved to file!

-- Processing: "_2016-11-23-18-49-13_exp3_Player.csv"
-- Retrieved 12 windows in _2016-11-23-18-49-13_exp3_Player.csv
Meta-features saved to file!

-- Processing: "_2016-11-23-18-49-13_exp4_Player.csv"
-- Retrieved 23 windows in _2016-11-23-18-49-13_exp4_Player.csv
Meta-features saved to file!

-- Processing: "_2016-11-23-18-49-13_exp5_Player.csv"
-- Retrieved 27 windows in _2016-11-23-18-49-13_exp5_Player.csv
Meta-features saved to file!

-- Processing: "_2016-11-24-15-43-37_exp1d_Player.csv"
-- Retrieved 31 windows in _2016-11-24-15-43-37_exp1d_Player.csv
Meta-features saved to file!

-- Processing: "_2016-11-24-15-43-37_exp2d_Player.csv"
-- Retrieved 48 windows in _2016-11-24-15-43-37_exp2d_P